<img src="../img/logo_amds.png" alt="Logo" style="width: 128px;"/>

# AmsterdamUMCdb - Freely Accessible ICU Database

version 1.0.2 March 2020  
Copyright &copy; 2003-2020 Amsterdam UMC - Amsterdam Medical Data Science

# Setup AmsterdamUMCdb
## Requirements
- Access to the AmsterdamUMCdb csv files: request access from [Amsterdam Medical Data Science](https://www.amsterdammedicaldatascience.nl/).
- Operating system: any OS capable of running Python and PostgreSQL, including Windows, macOS and Linux.
- Internal memory: 8GB should suffice for basic analysis and running the Jupyter notebooks. However, the recommended memory specification to run both PostgreSQL and the Jupyter Notebooks on the same machine is 16-32 GB.
- Disk space: Downloading and extracting the database files will require 90 GB of hard disk space. In addition, creating the PostgreSQL database requires about 128 GB of hard disk space and and an additional 144 GB for creating the indices to improve query performance. 

## 1. Install a Python distribution
We **strongly recommend** installing Python using Anaconda, a popular distribution that includes many useful modules for data science out-of-the-box. Install the (latest) Python 3.7 version distribution from [Anaconda's](https://www.anaconda.com/distribution) distribution page.

## 2. Install PostgreSQL
PostgreSQL is an open source database management system (DBMS), available for most operating systems, including Windows, macOS and Linux. We recommend the installation of the most recent version of PostgreSQL (version 12) from the PostgreSQL [download](https://www.postgresql.org/download/) page. Please note your password for the `postgres` superuser, and if you did not chose `postgres` as the password, you need to modify these settings in the [`config.SAMPLE.ini`](../config.SAMPLE.ini) file in the root of the repository. Save the file as [`config.ini`](../config.ini).

## 3. Install psycopg2 module
To connect to your postgreSQL server from Python, the [psycopg2](https://pypi.org/project/psycopg2/) package needs to be installed from the Anaconda Prompt/Shell using conda:

> conda install -c anaconda psycopg2

## 4. Clone the AmsterdamUMCdb GitHub respository
Clone or download the [AmsterdamUMCdb](https://github.com/AmsterdamUMC/AmsterdamUMCdb) repository from GitHub. 
Follow the instructions on GitHub's online step-by-step guide, if needed: https://help.github.com/en/github/creating-cloning-and-archiving-repositories/cloning-a-repository. 

## 5. Download the database files
Download the AmsterdamUMCdb zip file from and extract the files from the zip file to the [`data`](../data) folder of the cloned AmsterdamUMCdb repository.

## 6. Create database tables
Start the Jupyter notebook server from the command line (using Command Prompt on Windows or Terminal on Mac/Linux) by running

> jupyter notebook

From the Jupyter file browser, open the `setup-amsterdamumc.ipynb` file from the `setup-amsterdamumc` folder in the cloned repository. This code in the notebook assumes there is a default postgres installation with a dabase named `postgres`, user `postgres` with password `postgres`. You should change these settings in the [`config.SAMPLE.ini`](../config.SAMPLE.ini) file in the root of the repository and save the file as [`config.ini`](../config.ini).
To create the tables in the database run this Jupyter notebook, either cell by cell (▶️ Run) or use the ⏩ button to  perform all steps sequentially. An `amsterdamumc` [schema](https://www.postgresql.org/docs/12/ddl-schemas.html) will be created, and all tables will be added to this schema.

## 7. Verify the database
After this notebook has been run completely, the postgres database should contain all tables with the same number of records we released. The output should state `Verification: PASSED`. You can verify it [here](#verify).
 
## 8. Create database table indices
It's highly recommended to create some useful indices to improve performance for common queries on identifiers like admissionid, itemid and measured times. 

## 9. Jupyter Notebooks
While the indices are being created, the postgreSQL should be available for querying using the notebooks in the [`tables`](../tables) folder (with lower performance). We use  plotly (version >4) for interactive plots in some notebooks. Plotly can be installed by using conda:

> conda install -c plotly plotly

# Python settings
## Imports

In [5]:
%matplotlib inline
import psycopg2
import pandas as pd
import numpy as np

import io
import os
from IPython.display import display, HTML, Markdown, clear_output

from tqdm.auto import tqdm

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000

class ProgressFile:
    """ProgressFile: simple class to work as a file-like object that can be used by function to read the file
    while at the same time updating a progress bar.
    """
   
    def __init__(self, pbar, filename, mode="r"):
        self.file = open(filename, mode, encoding='windows-1252')
        self.progress = self.file.tell()
        self.pbar = pbar
        self.file.readline() #skip the first line: csv header

    def close(self):
        self.file.seek(0, os.SEEK_END)
        self.pbar.n = self.file.tell()
        self.pbar.refresh()
        self.file.close()
        
    def read(self, size):
        buf = self.file.read(size)
        self.pbar.update(size)

        return buf
        
    def readline(self, size):
        buf = self.file.readline(size)
        self.pbar.update(size)
        
        return buf
    
    
def copy_progress(csv, table):
    """copy the csv file to the table using tdqm progressbar.
    """
    #import the database using a tqdm progressbar and a ProgressFile
    pbar = tqdm(total=os.path.getsize(csv), desc='Importing '+os.path.splitext(os.path.basename(csv))[0],
            dynamic_ncols=True, unit_scale=1, unit='Bytes') #make a tdqm progress bar object

    pfile = ProgressFile(pbar, csv, 'r') #create a ProgressFile for showing progress
     
    cursor.copy_expert("""COPY {} FROM STDIN WITH (FORMAT CSV)""".format(table), pfile)
        
    #close the objects
    pfile.close()
    pbar.close()

    #show the first 10 records of this table
    df = pd.read_sql('SELECT * FROM ' + table + ' LIMIT 10',con)
    display(Markdown('## ' + os.path.splitext(os.path.basename(csv))[0] + ' (' + str(cursor.rowcount) + ' records copied):\n'))
    display(df)

## Connection settings

In [6]:
#Modify config.ini in the root folder of the repository to change the settings to connect to your postgreSQL database
import configparser
import os
config = configparser.ConfigParser()

if os.path.isfile('../config.ini'):
    config.read('../config.ini')
else:
    config.read('../config.SAMPLE.ini')

#Open a connection to the postgres database:
con = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'])
con.set_client_encoding('WIN1252') #Uses code page for Dutch accented characters.
con.set_session(autocommit=True)
cursor = con.cursor()

## Create schema
Create an `amsterdamumc` schema to prevent collissions with (possible) other tables in the default `public` schema and change the schema path to our newly created 'amsterdamumc' schema to access the data without schema qualifications (e.g. `admissions` instead of `amsterdamumc.admissions`.

In [7]:
sql = """
CREATE SCHEMA IF NOT EXISTS sepsis_amsterdamumcdb;
GRANT USAGE ON SCHEMA sepsis_amsterdamumcdb TO public;
GRANT CREATE ON SCHEMA sepsis_amsterdamumcdb TO public;
SET SCHEMA 'sepsis_amsterdamumcdb';
"""
cursor.execute(sql)

# Create admissions table

In [8]:
table = 'admissions'
sql = """
DROP TABLE IF EXISTS admissions CASCADE;
CREATE TABLE admissions 
(
    patientid INTEGER,
    admissionid serial PRIMARY KEY,
    admissioncount INTEGER,
    location VARCHAR,
    urgency BIT,
    origin VARCHAR,
    admittedat BIGINT,
    admissionyeargroup VARCHAR,
    dischargedat BIGINT,
    lengthofstay SMALLINT,
    destination VARCHAR,
    gender VARCHAR,
    agegroup VARCHAR,
    dateofdeath BIGINT,
    weightgroup VARCHAR,
    weightsource VARCHAR,
    heightgroup VARCHAR,
    heightsource VARCHAR,
    specialty VARCHAR
);
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_admissions (3136 records copied):


,patientid,admissionid,admissioncount,location,urgency,origin,admittedat,admissionyeargroup,dischargedat,lengthofstay,destination,gender,agegroup,dateofdeath,weightgroup,weightsource,heightgroup,heightsource,specialty
0,11,11,1,IC,1,None,0,2003-2009,148620000,42,Overleden,Man,60-69,1.334400e+08,80-89,Geschat,180-189,Geschat,Intensive Care Volwassenen
1,19,20,1,IC,1,None,0,2003-2009,605100000,168,16,None,60-69,NaN,70-79,None,None,None,Inwendig
2,24,25,1,MC,0,None,0,2003-2009,247020000,69,Overleden,Vrouw,70-79,1.887000e+08,80-89,Gemeten,160-169,Anamnestisch,Inwendig
3,41,44,1,IC,0,None,0,2003-2009,77220000,21,23,Vrouw,60-69,1.747296e+11,70-79,Anamnestisch,159-,Anamnestisch,Hematologie
4,43,47,2,IC,1,Verpleegafdeling zelfde ziekenhuis,166080000,2010-2016,562560000,110,25,Vrouw,60-69,3.300510e+10,70-79,Anamnestisch,160-169,Gemeten,Heelkunde Gastro-enterologie
5,46,50,1,IC,0,None,0,2003-2009,3158160000,878,Overleden,Vrouw,50-59,3.111360e+09,80-89,Anamnestisch,160-169,Anamnestisch,"Maag-,Darm-,Leverziekten"
6,54,58,1,IC&MC,1,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,340800000,95,25,Vrouw,50-59,5.300700e+09,59-,Anamnestisch,170-179,Anamnestisch,Heelkunde Gastro-enterologie
7,61,66,1,IC,0,None,0,2003-2009,1144080000,318,Overleden,Vrouw,60-69,1.071240e+09,59-,Anamnestisch,160-169,Anamnestisch,Heelkunde Gastro-enterologie
8,65,70,1,IC,0,None,0,2003-2009,112560000,32,16,Man,60-69,NaN,70-79,Geschat,180-189,Geschat,Heelkunde Gastro-enterologie
9,68,76,1,IC,1,Verpleegafdeling zelfde ziekenhuis,0,2010-2016,121260000,34,25,Man,40-49,NaN,110+,None,170-179,None,Heelkunde Gastro-enterologie


# Create drugitems table

In [9]:
table = 'drugitems'
sql = """
DROP TABLE IF EXISTS drugitems CASCADE;
CREATE TABLE drugitems 
(
    admissionid INTEGER,
    orderid BIGINT,
    ordercategoryid INTEGER,
    ordercategory VARCHAR,
    itemid INTEGER,
    item VARCHAR,
    isadditive BIT,
    isconditional BIT,
    rate FLOAT,
    rateunit VARCHAR,
    rateunitid INTEGER,
    ratetimeunitid INTEGER,
    doserateperkg BIT,
    dose FLOAT,
    doseunit VARCHAR,
    doserateunit VARCHAR,
    doseunitid INTEGER,    
    doserateunitid INTEGER,
    administered FLOAT,
    administeredunit VARCHAR,
    administeredunitid INTEGER,
    action VARCHAR,
    start BIGINT,
    stop BIGINT,
    duration BIGINT,
    solutionitemid INTEGER,
    solutionitem VARCHAR,
    solutionadministered FLOAT,
    solutionadministeredunit VARCHAR,
    fluidin FLOAT,
    iscontinuous BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_drugitems (1429070 records copied):


,admissionid,orderid,ordercategoryid,ordercategory,itemid,item,isadditive,isconditional,rate,rateunit,rateunitid,ratetimeunitid,doserateperkg,dose,doseunit,doserateunit,doseunitid,doserateunitid,administered,administeredunit,administeredunitid,action,start,stop,duration,solutionitemid,solutionitem,solutionadministered,solutionadministeredunit,fluidin,iscontinuous
0,11,1697064,15,Injecties Antimicrobiele middelen,7123,Imipenem (Tienam),0,0,0.0,None,None,None,0,500.0,mg,None,10,NaN,500.0,mg,10,Nieuwe toediening,97440000,97500000,1,8987.0,"NaCl 0,9% Medicatie",0.0,ml,0.0,0
1,11,1697456,15,Injecties Antimicrobiele middelen,7123,Imipenem (Tienam),0,0,0.0,None,None,None,0,500.0,mg,None,10,NaN,500.0,mg,10,Nieuwe toediening,126240000,126300000,1,8987.0,"NaCl 0,9% Medicatie",0.0,ml,0.0,0
2,11,1694998,15,Injecties Antimicrobiele middelen,12398,Levofloxacine (Tavanic),0,0,0.0,None,None,None,0,500.0,mg,None,10,NaN,500.0,mg,10,Nieuwe toediening,3060000,3120000,1,8992.0,Onverdund Medicatie,100.0,ml,100.0,0
3,11,1695837,15,Injecties Antimicrobiele middelen,12398,Levofloxacine (Tavanic),0,0,0.0,None,None,None,0,500.0,mg,None,10,NaN,500.0,mg,10,Nieuwe toediening,46260000,46320000,1,8992.0,Onverdund Medicatie,100.0,ml,100.0,0
4,11,1696764,15,Injecties Antimicrobiele middelen,12398,Levofloxacine (Tavanic),0,0,0.0,None,None,None,0,500.0,mg,None,10,NaN,500.0,mg,10,Nieuwe toediening,90240000,90300000,1,8992.0,Onverdund Medicatie,100.0,ml,100.0,0
5,11,1694915,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,3300000,6900000,60,NaN,None,NaN,None,500.0,0
6,11,1695118,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,1000.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,5820000,7620000,30,NaN,None,NaN,None,500.0,0
7,11,1695119,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,1000.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,9420000,11220000,30,NaN,None,NaN,None,500.0,0
8,11,1695126,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,13440000,17040000,60,NaN,None,NaN,None,500.0,0
9,11,1695350,17,Infuus - Colloid,7252,Gelofusine,0,0,NaN,None,None,None,0,500.0,ml,uur,6,5.0,500.0,ml,6,Nieuwe zak,23640000,27240000,60,NaN,None,NaN,None,500.0,0


# Create freetextitems table

In [10]:
table = 'freetextitems'
sql = """
DROP TABLE IF EXISTS freetextitems CASCADE;
CREATE TABLE freetextitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    value VARCHAR,
    comment VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_freetextitems (144801 records copied):


,admissionid,itemid,item,value,comment,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult
0,11,10900,Slijm (urine),Veel,None,-5760000,-5760000,Systeem,140870,None,1
1,11,10901,Bact. (urine),ZEER-VEE,None,-5760000,-5760000,Systeem,140900,None,1
2,11,11636,Ery's (urine),0-5,None,-5760000,-5760000,Systeem,140793,None,1
3,11,11644,Rond-Epith (urine),Veel,None,-5760000,-5760000,Systeem,140840,None,1
4,11,11646,Afname (bloed),ART.,None,-6540000,-6540000,Systeem,140230,None,1
5,11,11646,Afname (bloed),ART.,None,-4200000,-4200000,Systeem,141026,None,1
6,20,10780,VIRUSkweek (sputum),volgt,None,217020000,217020000,Systeem,237387404,None,1
7,20,10866,RSV IF (sputum),ZIE-OPM,virussen.`Niet te beoordelen,217020000,217020000,Systeem,300391340,None,1
8,20,10892,Ketonen (urine),NEGATIEF,None,35340000,35340000,Systeem,222523107,None,1
9,20,10898,Type Cyl. (urine),HYAL.CYL,None,-1800000,-1800000,Systeem,234936,None,1


# Create listitems table

In [11]:
table = 'listitems'
sql = """
DROP TABLE IF EXISTS listitems CASCADE;
CREATE TABLE listitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    valueid INT,
    value VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_listitems (7121410 records copied):


,admissionid,itemid,item,valueid,value,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult
0,11,6671,Hartritme,21,Sinus Tac,3840000,5760000,ICV_IC-Verpleegkundig,5760000,ICV_IC-Verpleegkundig,0
1,11,6671,Hartritme,21,Sinus Tac,5040000,11640000,ICV_IC-Verpleegkundig,11640000,ICV_IC-Verpleegkundig,0
2,11,6671,Hartritme,21,Sinus Tac,7440000,11580000,ICV_IC-Verpleegkundig,11580000,ICV_IC-Verpleegkundig,0
3,11,6671,Hartritme,21,Sinus Tac,11040000,11580000,ICV_IC-Verpleegkundig,11580000,ICV_IC-Verpleegkundig,0
4,11,6671,Hartritme,21,Sinus Tac,14640000,14700000,ICV_IC-Verpleegkundig,14700000,ICV_IC-Verpleegkundig,0
5,11,6671,Hartritme,21,Sinus Tac,18240000,18480000,ICV_IC-Verpleegkundig,18480000,ICV_IC-Verpleegkundig,0
6,11,6671,Hartritme,21,Sinus Tac,21840000,24540000,ICV_IC-Verpleegkundig,24540000,ICV_IC-Verpleegkundig,0
7,11,6671,Hartritme,21,Sinus Tac,23640000,28380000,ICV_IC-Verpleegkundig,28380000,ICV_IC-Verpleegkundig,0
8,11,6671,Hartritme,21,Sinus Tac,25440000,28380000,ICV_IC-Verpleegkundig,28380000,ICV_IC-Verpleegkundig,0
9,11,6671,Hartritme,21,Sinus Tac,29040000,29460000,ICV_IC-Verpleegkundig,29460000,ICV_IC-Verpleegkundig,0


# Create numericitems table
This is the largest table and can take a while depending on the performance of your system.

In [12]:
table = 'numericitems'
sql = """
    DROP TABLE IF EXISTS numericitems CASCADE;
    CREATE TABLE numericitems 
    (
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    tag VARCHAR,
    value FLOAT,
    unitid INT,
    unit VARCHAR,
    comment VARCHAR,
    measuredat BIGINT,
    registeredat BIGINT,
    registeredby VARCHAR,
    updatedat BIGINT,
    updatedby VARCHAR,
    islabresult BIT,
    fluidout FLOAT
    )
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_numericitems (194838488 records copied):


,admissionid,itemid,item,tag,value,unitid,unit,comment,measuredat,registeredat,registeredby,updatedat,updatedby,islabresult,fluidout
0,11,6640,Hartfrequentie,,150.0,15,/min,Opmerking verwijderd,3840000,5700000,ICV_IC-Verpleegkundig,5700000,ICV_IC-Verpleegkundig,0,None
1,11,6640,Hartfrequentie,,143.0,15,/min,Opmerking verwijderd,5040000,7800000,ICV_IC-Verpleegkundig,7800000,ICV_IC-Verpleegkundig,0,None
2,11,6640,Hartfrequentie,,147.0,15,/min,Opmerking verwijderd,7440000,7800000,ICV_IC-Verpleegkundig,7800000,ICV_IC-Verpleegkundig,0,None
3,11,6640,Hartfrequentie,,145.0,15,/min,Opmerking verwijderd,11040000,11580000,ICV_IC-Verpleegkundig,11580000,ICV_IC-Verpleegkundig,0,None
4,11,6640,Hartfrequentie,,141.0,15,/min,Opmerking verwijderd,14640000,14700000,ICV_IC-Verpleegkundig,14700000,ICV_IC-Verpleegkundig,0,None
5,11,6640,Hartfrequentie,,133.0,15,/min,Opmerking verwijderd,18240000,18480000,ICV_IC-Verpleegkundig,18480000,ICV_IC-Verpleegkundig,0,None
6,11,6640,Hartfrequentie,,141.0,15,/min,Opmerking verwijderd,21840000,24480000,ICV_IC-Verpleegkundig,24480000,ICV_IC-Verpleegkundig,0,None
7,11,6640,Hartfrequentie,,135.0,15,/min,Opmerking verwijderd,23640000,24780000,ICV_IC-Verpleegkundig,24780000,ICV_IC-Verpleegkundig,0,None
8,11,6640,Hartfrequentie,,131.0,15,/min,Opmerking verwijderd,25440000,28380000,ICV_IC-Verpleegkundig,28380000,ICV_IC-Verpleegkundig,0,None
9,11,6640,Hartfrequentie,,126.0,15,/min,Opmerking verwijderd,29040000,29460000,ICV_IC-Verpleegkundig,29460000,ICV_IC-Verpleegkundig,0,None


# Create procedureorderitems table

In [13]:
table = 'procedureorderitems'
sql = """
DROP TABLE IF EXISTS procedureorderitems CASCADE;
CREATE TABLE procedureorderitems 
(
    admissionid INTEGER,
    orderid BIGINT,
    ordercategoryid INT,
    ordercategoryname VARCHAR,
    itemid INT,
    item VARCHAR,
    registeredat BIGINT,
    registeredby VARCHAR
)
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_procedureorderitems (580232 records copied):


,admissionid,orderid,ordercategoryid,ordercategoryname,itemid,item,registeredat,registeredby
0,11,1695373,73,Opdr. Laboratorium afname,9250,CVVH-lab. afnemen,25320000,ICV_IC-Verpleegkundig
1,11,1696309,73,Opdr. Laboratorium afname,9250,CVVH-lab. afnemen,73440000,ICV_IC-Verpleegkundig
2,11,1697172,73,Opdr. Laboratorium afname,9250,CVVH-lab. afnemen,112860000,ICV_IC-Verpleegkundig
3,11,1696308,73,Opdr. Laboratorium afname,9446,Lab. dgs Standaard 6.00u,73440000,ICV_IC-Verpleegkundig
4,11,1695071,73,Opdr. Laboratorium afname,10605,HI-schema T=0,10920000,ICV_IC-Verpleegkundig
5,11,1695374,73,Opdr. Laboratorium afname,10606,HI-schema T=6,25320000,ICV_IC-Verpleegkundig
6,11,1695872,73,Opdr. Laboratorium afname,12392,HI-schema T=12,49260000,ICV_IC-Verpleegkundig
7,11,1696314,73,Opdr. Laboratorium afname,12393,HI-schema T=18,73560000,ICV_IC-Verpleegkundig
8,11,1696747,73,Opdr. Laboratorium afname,12394,HI-schema T=24,92280000,ICV_IC-Verpleegkundig
9,11,1694999,74,Opdr. Kweken afnemen,9205,Legionella sneltest (urine),6480000,ICV_IC-Verpleegkundig


# Create processitems table

In [14]:
table = 'processitems'
sql = """
DROP TABLE IF EXISTS processitems CASCADE;
CREATE TABLE processitems 
(
    admissionid INTEGER,
    itemid BIGINT,
    item VARCHAR,
    start BIGINT,
    stop BIGINT,
    duration BIGINT
);
"""
cursor.execute(sql)

csv = os.path.join('..', config['files']['datapath'], 'sepsis_' + config['files'][table])
copy_progress(csv, table) #runs copy_from using a tdqm progress bar

## sepsis_processitems (44405 records copied):


,admissionid,itemid,item,start,stop,duration
0,11,7927,Buikligging,72240000,102660000,507
1,11,9159,Arterielijn Radialis,3840000,132840000,2150
2,11,9166,Trilumen Jugularis,3840000,132840000,2150
3,11,9328,Beademen,3840000,132840000,2150
4,11,9415,Urine-Catheter (CAD),3840000,132840000,2150
5,11,9417,Sonde,3840000,72240000,1140
6,11,9417,Sonde,72240000,132840000,1010
7,11,9422,Perifeer infuus,3840000,132840000,2150
8,11,12465,CVVH,15000000,43440000,474
9,11,12465,CVVH,52440000,100440000,800


## Create Indices to increase performance
After verification this will run to create indices to improve query performance. However, this is a height process that can take hours depending on your system. In the meantime, the database is already to be queried (albeit slower), using the notebooks from the [tables](../tables/) folder.

In [15]:
import select
import psycopg2.extensions
from tqdm.notebook import tqdm
#from tqdm.notebook import tqdm
import time

def wait(conn):
    while True:
        state = conn.poll()
        if state == psycopg2.extensions.POLL_OK:
            break
        elif state == psycopg2.extensions.POLL_WRITE:
            select.select([], [conn.fileno()], [])
        elif state == psycopg2.extensions.POLL_READ:
            select.select([conn.fileno()], [], [])
        else:
            raise psycopg2.OperationalError("poll() returned %s" % state)

def get_status():
    status_sql = """
    SELECT c.relname as tablename, s.*, a.query FROM pg_stat_progress_create_index s 
    JOIN pg_stat_activity a ON s.pid = a.pid
    JOIN pg_class c on s.relid = c.oid
    WHERE query LIKE '%-- sepsis_amsterdamumcdb indices%';
    """
    status = pd.read_sql(status_sql, con)
    if len(status) > 0:
        pid = status['pid'][0]
        phase = status['phase'][0]
        tablename = status['tablename'][0]
        
        if 'tuples' in phase:
            total = status['tuples_total'][0]
            current = status['tuples_done'][0]
        else:
            total = status['blocks_total'][0]
            current = status['blocks_done'][0]

        return pid, total, current, phase, tablename
    else:
        #no running index available
        return None, None, None, None, None
    

#create a new async connection, to monitor the status from another connection while the indexing is running.
aconn = psycopg2.connect(database=config['psycopg2']['database'], 
                       user=config['psycopg2']['username'], password=config['psycopg2']['password'], 
                       host=config['psycopg2']['host'], port=config['psycopg2']['port'], async_=1) 

wait(aconn) #wait until connection is ready
acurs = aconn.cursor()

index_sql = """
-- sepsis_amsterdamumcdb indices
SET SCHEMA 'sepsis_amsterdamumcdb';
-- admissions table
CREATE INDEX admissions_admissionid_index ON admissions (admissionid);
CREATE INDEX admissions_patientid_index ON admissions (patientid);

-- drugitems table
CREATE INDEX drugitems_admissionid_index ON drugitems (admissionid);
CREATE INDEX drugitems_orderid_index ON drugitems (orderid);
CREATE INDEX drugitems_ordercategoryid_index ON drugitems (ordercategoryid);
CREATE INDEX drugitems_itemid_index ON drugitems (itemid);
CREATE INDEX drugitems_start_index ON drugitems (start);
CREATE INDEX drugitems_stop_index ON drugitems (stop);

-- freetextitems table
CREATE INDEX freetextitems_admissionid_index ON freetextitems (admissionid);
CREATE INDEX freetextitems_itemid_index ON freetextitems (itemid);
CREATE INDEX freetextitems_measuredat_index ON freetextitems (measuredat);

-- listitems table
CREATE INDEX listitems_admissionid_index ON listitems (admissionid);
CREATE INDEX listitems_itemid_index ON listitems (itemid);
CREATE INDEX listitems_measuredat_index ON listitems (measuredat);

-- numericitems table
CREATE INDEX numericitems_admissionid_index ON numericitems (admissionid);
CREATE INDEX numericitems_itemid_index ON numericitems (itemid);
CREATE INDEX numericitems_measuredat_index ON numericitems (measuredat);
CREATE INDEX numericitems_admission_item_time_index ON numericitems (admissionid, itemid, measuredat);
CREATE INDEX numericitems_islabresult_index ON numericitems (islabresult);
CREATE INDEX numericitems_fluidout_index ON numericitems (fluidout);

-- procedureorderitems table
CREATE INDEX procedureorderitems_admissionid_index ON procedureorderitems (admissionid);
CREATE INDEX procedureorderitems_itemid_index ON procedureorderitems (itemid);
CREATE INDEX procedureorderitems_ordercategoryid_index ON procedureorderitems (ordercategoryid);
CREATE INDEX procedureorderitems_registeredat_index ON procedureorderitems (registeredat);

-- processitems table
CREATE INDEX processitems_admissionid_index ON processitems (admissionid);
CREATE INDEX processitems_itemid_index ON processitems (itemid);
CREATE INDEX processitems_start_index ON processitems (start);
CREATE INDEX processitems_stop_index ON processitems (stop);
"""
#execute create index sql query **asychronously**
acurs.execute(index_sql)

#wait 2 seconds to allow for starting up the indexing process 
time.sleep(2)

#progress bar variables
progress_blocks = None
progress_tuples = None
progress_current = None

#used for progress of indexing the database (based on indices in release 1.0.2)
TOTAL_INDEX_BLOCKS = 97361839
TOTAL_INDEX_TUPLES = 5.999210e+09

#get the current status of the indexing operation

pid, total, current, previous_phase, previous_tablename = get_status()
if pid == None:
    print('No indexing in progress.')
else:

    #create two tqdm objects for progress bars
    progress_blocks = tqdm(total=TOTAL_INDEX_BLOCKS, desc='Total blocks read', dynamic_ncols=True, 
                           unit_scale=1, unit='blocks') #total blocks progress
    progress_tuples = tqdm(total=TOTAL_INDEX_TUPLES, desc='Total tuples written', dynamic_ncols=True, 
                           unit_scale=1, unit='tuples') #total tuples progress
    progress_current = tqdm(total=total, desc='Processing table ' + previous_tablename + ' (' + previous_phase + ')',
                            dynamic_ncols=True, unit_scale=1, leave=False) #current index progress

    #update the progress bars every two seconds until the progress query returns empty or changes
    while True:  
            current_pid, total, current, phase, tablename = get_status()

            if pid == None or current_pid != pid:
                break #process changed: done

            #phase changed, if 'scanning table' a new item was started
            if (not previous_phase == phase) or \
                (not previous_tablename == tablename):
                
                reset = False
                if 'scanning table' in phase: #scanning tables, loading tuples in tree
                    #previous phase ('tuples') should be added completely to progress bar
                    progress_tuples.update(progress_current.total - progress_current.n)
                    reset = True
                    
                elif 'loading tuples in tree' in phase: #loading tuples in tree
                    #previous phase ('scanning table') should be added completely to progress bar
                    progress_blocks.update(progress_current.total - progress_current.n)          
                    reset = True
                
                if reset:
                    #finalize (100%) the progress bar, remove it and start a new one
                    progress_current.n = progress_current.total
                    progress_current.close()
                    progress_current = tqdm(total=total, desc='Processing table ' + tablename + ' (' + phase + ')',
                                            dynamic_ncols=True, unit_scale=1, leave=False) #current index progress
            
            #update progress bars based on the phase
            update = False
            if 'scanning table' in phase: #scanning tables, loading tuples in tree

                progress_current.unit = 'blocks'

                #update the progress of total blocks to read
                progress_blocks.update(current - progress_current.n)
                update = True

            elif 'loading tuples in tree' in phase:
                progress_current.unit = 'tuples'

                #update the progress of total tuples written
                progress_tuples.update(current - progress_current.n)
                update = True

            if update:
                #update the progress of the current index
                progress_current.n = current
                progress_current.update(0) #trigger refresh

                #store the current phase to compare with the next
                previous_phase = phase
                previous_tablename = tablename

            
            time.sleep(1) #wait for one second before requerying indexing progress

    #left while loop
    if not progress_blocks is None:
        progress_blocks.n = TOTAL_INDEX_BLOCKS
        progress_blocks.close()
    if not progress_tuples is None:
        progress_tuples.n = TOTAL_INDEX_TUPLES
        progress_tuples.close()
    if not progress_current is None:
        progress_current.n = progress_current.total
        progress_current.close()
        
    print('Indexing done.')
    
wait(aconn) #wait until connection is ready
aconn.close()



Indexing done.
